# Análisis de Datos Turísticos
Análisis y visualización de datos del Registro Nacional de Turismo

In [4]:
import micropip

await micropip.install('ipywidgets')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

In [5]:
def leer_negocios(archivo_csv):
    try:
        df = pd.read_csv(archivo_csv, low_memory=False)
        return df
    except Exception as e:
        print(f"Error al leer el archivo: {str(e)}")
        return None

# Load the data
archivo_csv = "./Registro_Nacional_de_Turismo_-_RNT_20250328.csv"
df = leer_negocios(archivo_csv)

In [6]:
def filtrar_datos(df, departamento=None, municipio=None, categorias=None, sub_categoria=None):
    df_filtrado = df.copy()
    
    if departamento:
        df_filtrado = df_filtrado[df_filtrado['DEPARTAMENTO'].str.upper() == departamento.upper()]
    
    if municipio:
        df_filtrado = df_filtrado[df_filtrado['MUNICIPIO'].str.upper() == municipio.upper()]
    
    if categorias:
        df_filtrado = df_filtrado[df_filtrado['CATEGORIA'].str.upper().isin([cat.upper() for cat in categorias])]
    if sub_categoria:
        df_filtrado = df_filtrado[df_filtrado['CATEGORIA'].str.upper().isin([cat.upper() for cat in categorias])]
    
    return df_filtrado

In [13]:
# Interactive filters
from ipywidgets import interact, widgets

departamentos = sorted(df['DEPARTAMENTO'].unique())
categorias = sorted(df['CATEGORIA'].unique())
# municipio = sorted(df['MUNICIPIO'].unique())
sub_categoria = sorted(df['SUB_CATEGORIA'].unique())

# Filter for accommodation establishments
df = df[df['CATEGORIA'] == 'ESTABLECIMIENTOS DE ALOJAMIENTO TURÍSTICO']
@interact
def actualizar_filtros(departamento=widgets.Dropdown(options=[''] + list(departamentos), description='Departamento:'),
                       categoria=widgets.Dropdown(options=[''] + list(categorias), description='Categoría:'),
                      sub_categoria=widgets.Dropdown(options=[''] + list(sub_categoria), description='Sub Categoría:')
                      ):
    municipio = sorted(df[df['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique())
    @interact
    def update_municipality(municipio=widgets.Dropdown(options=municipio,
                                                     description='Municipality:',
                                                     style={'description_width': 'initial'})):
    
    # Filter data
        filtered_df = filtrar_datos(df,departamento if departamento else None,municipio if municipio else None,[categoria] if categoria else None)
        
        # Display basic statistics
        print(f"Total establecimientos: {len(filtered_df)}")
        
        # Create visualizations
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot 1: Top 10 municipalities
        municipios_count = filtered_df['MUNICIPIO'].value_counts().head(10)
        municipios_count.plot(kind='bar', ax=ax1)
        ax1.set_title('Top 10 Municipios')
        ax1.tick_params(axis='x', rotation=45)
        
        # Plot 2: Categories distribution
        categorias_count = filtered_df['SUB_CATEGORIA'].value_counts().head(10)
        categorias_count.plot(kind='bar', ax=ax2)
        ax2.set_title('Top 10 Sub-Categorías')
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Display detailed information
        return filtered_df[['RAZON_SOCIAL_ESTABLECIMIENTO', 'DEPARTAMENTO', 'MUNICIPIO', 
                        'CATEGORIA', 'SUB_CATEGORIA', 'HABITACIONES', 'CAMAS', 'CORREO_ESTABLECIMIENTO']]

interactive(children=(Dropdown(description='Departamento:', options=('', 'AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'A…

In [8]:
def analizar_hoteles_municipio(departamento):
    # Filter by department
    df_dept = df[df['DEPARTAMENTO'] == departamento]
    
    # Count establishments by municipality
    hoteles_por_municipio = df_dept.groupby('MUNICIPIO').size().reset_index(name='NUMERO_HOTELES')
    
    # Sort by number of hotels descending
    hoteles_por_municipio = hoteles_por_municipio.sort_values('NUMERO_HOTELES', ascending=False)
    
    # Create figure
    plt.figure(figsize=(15, 8))
    
    # Create bar plot
    sns.barplot(data=hoteles_por_municipio, 
                x='MUNICIPIO', 
                y='NUMERO_HOTELES',
                color='coral')
    
    plt.title(f'Number of Hotels by Municipality in {departamento}')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Number of Hotels')
    plt.xlabel('Municipality')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Total Municipalities: {len(hoteles_por_municipio)}")
    print(f"Total Hotels: {hoteles_por_municipio['NUMERO_HOTELES'].sum():,.0f}")
    print("\nTop 5 Municipalities by Number of Hotels:")
    print(hoteles_por_municipio.head().to_string(index=False))
    
    return hoteles_por_municipio

# Create interactive widget

departamentos = sorted(df['DEPARTAMENTO'].unique())
interact(analizar_hoteles_municipio, 
         departamento=widgets.Dropdown(options=departamentos, 
                                     description='Department:',
                                     style={'description_width': 'initial'}))

interactive(children=(Dropdown(description='Department:', options=('AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTI…

<function __main__.analizar_hoteles_municipio(departamento)>

In [9]:
df['HABITACIONES'] = pd.to_numeric(df['HABITACIONES'], errors='coerce')
df['CAMAS'] = pd.to_numeric(df['CAMAS'], errors='coerce')
def analizar_departamento(departamento):
    # Filter by department
    df_dept = df[df['DEPARTAMENTO'] == departamento]
    
    # Group by municipality and calculate totals
    municipios_stats = df_dept.groupby('MUNICIPIO').agg({
        'HABITACIONES': 'sum',
        'CAMAS': 'sum'
    }).reset_index()
    
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # Plot 1: Rooms by Municipality
    sns.barplot(data=municipios_stats, x='MUNICIPIO', y='HABITACIONES', ax=ax1, color='skyblue')
    ax1.set_title(f'Total Rooms by Municipality in {departamento}')
    ax1.tick_params(axis='x', rotation=45)
    ax1.set_ylabel('Number of Rooms')
    
    # Plot 2: Beds by Municipality
    sns.barplot(data=municipios_stats, x='MUNICIPIO', y='CAMAS', ax=ax2, color='lightgreen')
    ax2.set_title(f'Total Beds by Municipality in {departamento}')
    ax2.tick_params(axis='x', rotation=45)
    ax2.set_ylabel('Number of Beds')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nSummary Statistics:")
    print(f"Total Municipalities: {len(municipios_stats)}")
    print(f"Total Rooms: {municipios_stats['HABITACIONES'].sum():,.0f}")
    print(f"Total Beds: {municipios_stats['CAMAS'].sum():,.0f}")
    
    return municipios_stats


departamentos = sorted(df['DEPARTAMENTO'].unique())
interact(analizar_departamento, 
         departamento=widgets.Dropdown(options=departamentos, 
                                     description='Department:',
                                     style={'description_width': 'initial'}))

interactive(children=(Dropdown(description='Department:', options=('AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTI…

<function __main__.analizar_departamento(departamento)>

In [ ]:
def analizar_hoteles_detallado(departamento, municipio, min_rooms=0, max_rooms=1000):
    # Filter by department and municipality
    df_local = df[(df['DEPARTAMENTO'] == departamento) & 
                  (df['MUNICIPIO'] == municipio)]
    
    # Additional filter for room range
    df_local = df_local[(df_local['HABITACIONES'] >= min_rooms) & 
                        (df_local['HABITACIONES'] <= max_rooms)]
    
    if len(df_local) == 0:
        print(f"No hotels found in {municipio}, {departamento} with {min_rooms}-{max_rooms} rooms")
        return
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # Sort by number of rooms
    df_sorted_rooms = df_local.sort_values('HABITACIONES', ascending=True)
    
    # Plot 1: Rooms by Hotel
    sns.barplot(data=df_sorted_rooms, 
                y='RAZON_SOCIAL_ESTABLECIMIENTO', 
                x='HABITACIONES',
                ax=ax1,
                color='skyblue')
    ax1.set_title(f'Hotels with {min_rooms}-{max_rooms} Rooms in {municipio}, {departamento}')
    ax1.set_xlabel('Number of Rooms')
    ax1.set_ylabel('Hotel Name')
    
    # Plot 2: Beds by Hotel
    sns.barplot(data=df_sorted_rooms, 
                y='RAZON_SOCIAL_ESTABLECIMIENTO', 
                x='CAMAS',
                ax=ax2,
                color='lightgreen')
    ax2.set_title(f'Number of Beds (Hotels with {min_rooms}-{max_rooms} Rooms)')
    ax2.set_xlabel('Number of Beds')
    ax2.set_ylabel('Hotel Name')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("\nDetailed Statistics:")
    print(f"Hotels in range {min_rooms}-{max_rooms} rooms: {len(df_local)}")
    print(f"Total Rooms: {df_local['HABITACIONES'].sum():,.0f}")
    print(f"Total Beds: {df_local['CAMAS'].sum():,.0f}")
    print(f"Average Rooms per Hotel: {df_local['HABITACIONES'].mean():,.1f}")
    print(f"Average Beds per Hotel: {df_local['CAMAS'].mean():,.1f}")
    
    print("\nHotel Details:")
    hotel_details = df_local[['RAZON_SOCIAL_ESTABLECIMIENTO', 'HABITACIONES', 'CAMAS', 'NUM_EMP']]
    print(hotel_details.to_string(index=False))


departamentos = sorted(df['DEPARTAMENTO'].unique())

@interact
def select_location(departamento=widgets.Dropdown(options=departamentos, 
                                                  description='Department:', 
                                                  style={'description_width': 'initial'})
                    ):
    municipios = sorted(df[df['DEPARTAMENTO'] == departamento]['MUNICIPIO'].unique())
    
    @interact
    def update_municipality(municipio=widgets.Dropdown(options=municipios,
                                                     description='Municipality:',
                                                     style={'description_width': 'initial'}),
                          min_rooms=widgets.IntSlider(min=0, max=100, step=1, value=0,
                                                    description='Min Rooms:'),
                          max_rooms=widgets.IntSlider(min=0, max=100, step=1, value=50,
                                                    description='Max Rooms:')):
        analizar_hoteles_detallado(departamento, municipio, min_rooms, max_rooms)

interactive(children=(Dropdown(description='Department:', options=('AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTI…